In [54]:
import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import pandas as pd
import numpy as np

import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix

In [56]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
data=pd.read_csv(r"C:\Users\hp\Desktop\spam_ham_dataset.csv")
data

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [59]:
data['text']=data['text'].apply(lambda x:x.replace('\r\n',' '))

In [60]:
data=data.drop('Unnamed: 0',axis=1)

In [64]:
data.head(10)

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291 thi...,0
1,ham,"Subject: hpl nom for january 9 , 2001 ( see at...",0
2,ham,"Subject: neon retreat ho ho ho , we ' re aroun...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs this deal is to b...,0
5,ham,Subject: ehronline web address change this mes...,0
6,ham,Subject: spring savings certificate - take 30 ...,0
7,spam,Subject: looking for medication ? we ` re the ...,1
8,ham,Subject: noms / actual flow for 2 / 26 we agre...,0
9,ham,"Subject: nominations for oct . 21 - 23 , 2000 ...",0


In [66]:
data.tail(10)

,label,text,label_num
5161,spam,Subject: penny stocks are about timing nomad i...,1
5162,spam,Subject: anomaly boys from 3881 uosda apaprove...,1
5163,ham,Subject: re : meter # : 1266 ; august 2000 / a...,0
5164,spam,Subject: slutty milf wants to meet you take th...,1
5165,ham,"Subject: fw : crosstex energy , driscoll ranch...",0
5166,ham,Subject: put the 10 on the ft the transport vo...,0
5167,ham,Subject: 3 / 4 / 2000 and following noms hpl c...,0
5168,ham,Subject: calpine daily gas nomination > > juli...,0
5169,ham,Subject: industrial worksheets for august 2000...,0
5170,spam,Subject: important online banking alert dear v...,1


In [68]:
data.describe()

,label_num
count,5171.000000
mean,0.289886
std,0.453753
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      5171 non-null   object
 1   text       5171 non-null   object
 2   label_num  5171 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 121.3+ KB


Data Preprocessing

In [73]:
stemmer=PorterStemmer()
corpus=[]

stopwords_set=set(stopwords.words('english'))

for i in range(len(data)):
    text=data['text'].iloc[i].lower()
    text=text.translate(str.maketrans('','',string.punctuation)).split()
    text=[stemmer.stem(word) for word in text if word not in stopwords_set]
    text=' '.join(text)
    corpus.append(text)

Applying Count Vectorizer 

In [75]:
vectorizer=CountVectorizer()
X=vectorizer.fit_transform(corpus).toarray()
y=data.label_num

Splitting data

In [98]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [99]:
NB_classifier=MultinomialNB()
NB_classifier.fit(X_train,y_train)

MultinomialNB()

In [100]:
NB_classifier.score(X_test,y_test)

0.9777777777777777

Evaluating the model

In [102]:
y_predict_train=NB_classifier.predict(X_train)
y_predict_train

array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

In [103]:
cm=confusion_matrix(y_train,y_predict_train)

In [105]:
y_predict_test=NB_classifier.predict(X_test)
y_predict_test

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [106]:
cm=confusion_matrix(y_test,y_predict_test)

In [113]:
print(classification_report(y_test,y_predict_test))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98       732
           1       0.95      0.98      0.96       303

    accuracy                           0.98      1035
   macro avg       0.97      0.98      0.97      1035
weighted avg       0.98      0.98      0.98      1035

